In [5]:
import pandas as pd
import numpy as np
import unicodedata
import re

from bs4 import BeautifulSoup
import requests
import os
import json

import nltk
from nltk.corpus import stopwords

In [30]:
df = pd.read_csv('first_525.csv', index_col=0)

In [31]:
df

,title,summary,year_published,author,review_count,number_of_ratings,length,genre,rating,reviews
0,Harry Potter and the Deathly Hallows,"Harry has been burdened with a dark, dangerous...",2007,J.K. Rowling,78713,3438896,759.0,Fantasy,4.62,"[""Ok, before I start a few warnings. This will..."
1,The Hunger Games,"Could you survive on your own in the wild, wit...",2008,Suzanne Collins,192474,7880793,374.0,Young Adult,4.33,"['LMAAAOOOO! Thats Peeta folks! xDHahahahaha, ..."
2,The Kite Runner,1970s Afghanistan: Twelve-year-old Amir is des...,2003,Khaled Hosseini,90066,2930607,371.0,Fiction,4.33,['Finished this book about a month ago but it\...
3,The Book Thief,Librarian's note: An alternate cover edition c...,2006,Markus Zusak,134599,2340598,552.0,Historical Fiction,4.39,"[""Just to clarify: Yes, I did cry.I've read a ..."
4,Harry Potter and the Half-Blood Prince,"It is the middle of the summer, but there is a...",2005,J.K. Rowling,51324,3021148,652.0,Fantasy,4.58,['(A) 86% | ExtraordinaryNotes: It dwells on t...
...,...,...,...,...,...,...,...,...,...,...
120,A Court of Mist and Fury,Feyre survived Amarantha's clutches to return ...,2016,Sarah J. Maas,90174,1008670,626.0,Fantasy,4.63,['HOLY FUCKING SHIT. HOLY. FUCKING. SHIT.A COU...
121,Iron Kissed,"I could smell her fear, and it satisfied somet...",2008,Patricia Briggs,4407,119969,287.0,Urban Fantasy,4.34,"[""I fucking cried like a big baby made of baby..."
122,Bossypants,"Before Liz Lemon, before ""Weekend Update,"" bef...",2011,Tina Fey,41347,962005,283.0,Nonfiction,3.96,"[""Tina Fey's Bossypants was a disappointment. ..."
123,Peony in Love,"In seventeenth-century China, three women beco...",2007,Lisa See,4749,46317,273.0,Historical Fiction,3.63,"[""5/1/14 EDIT: Sooo I stopped checking comment..."


In [42]:
df.nunique()

title                522
summary              521
year_published        31
author               354
review_count         518
number_of_ratings    522
length               308
genre                 27
rating               117
reviews              522
dtype: int64

In [49]:
df = df.drop_duplicates(subset='title')

In [50]:
df

,title,summary,year_published,author,review_count,number_of_ratings,length,genre,rating,reviews
0,Harry Potter and the Deathly Hallows,"Harry has been burdened with a dark, dangerous...",2007,J.K. Rowling,78713,3438896,759.0,Fantasy,4.62,"[""Ok, before I start a few warnings. This will..."
1,The Hunger Games,"Could you survive on your own in the wild, wit...",2008,Suzanne Collins,192474,7880793,374.0,Young Adult,4.33,"['LMAAAOOOO! Thats Peeta folks! xDHahahahaha, ..."
2,The Kite Runner,1970s Afghanistan: Twelve-year-old Amir is des...,2003,Khaled Hosseini,90066,2930607,371.0,Fiction,4.33,['Finished this book about a month ago but it\...
3,The Book Thief,Librarian's note: An alternate cover edition c...,2006,Markus Zusak,134599,2340598,552.0,Historical Fiction,4.39,"[""Just to clarify: Yes, I did cry.I've read a ..."
4,Harry Potter and the Half-Blood Prince,"It is the middle of the summer, but there is a...",2005,J.K. Rowling,51324,3021148,652.0,Fantasy,4.58,['(A) 86% | ExtraordinaryNotes: It dwells on t...
...,...,...,...,...,...,...,...,...,...,...
120,A Court of Mist and Fury,Feyre survived Amarantha's clutches to return ...,2016,Sarah J. Maas,90174,1008670,626.0,Fantasy,4.63,['HOLY FUCKING SHIT. HOLY. FUCKING. SHIT.A COU...
121,Iron Kissed,"I could smell her fear, and it satisfied somet...",2008,Patricia Briggs,4407,119969,287.0,Urban Fantasy,4.34,"[""I fucking cried like a big baby made of baby..."
122,Bossypants,"Before Liz Lemon, before ""Weekend Update,"" bef...",2011,Tina Fey,41347,962005,283.0,Nonfiction,3.96,"[""Tina Fey's Bossypants was a disappointment. ..."
123,Peony in Love,"In seventeenth-century China, three women beco...",2007,Lisa See,4749,46317,273.0,Historical Fiction,3.63,"[""5/1/14 EDIT: Sooo I stopped checking comment..."


In [54]:
#Define function to clean up text data
def clean_text(text, extra_stopwords=['r']):
    
    wnl = nltk.stem.WordNetLemmatizer()
    
    stopwords = nltk.corpus.stopwords.words('english') + extra_stopwords
    
    clean_text = (unicodedata.normalize('NFKD', text)
                   .encode('ascii', 'ignore')
                   .decode('utf-8', 'ignore')
                   .lower())
    
    words = re.sub(r'[^\w\s]', '', clean_text).split()
    
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [55]:
df['clean_summary'] = clean_text(' '.join(df['summary']))


TypeError: sequence item 481: expected str instance, float found

In [ ]:
ham_counts = pd.Series(ham_words).value_counts()


In [ ]:
spam_counts.head()

In [ ]:
word_freq = pd.concat([ham_counts, spam_counts, all_counts], axis=1)


In [ ]:
word_freq.isna().sum()

In [ ]:
word_freq.columns = ['ham', 'spam', 'all']
word_freq.fillna(0, inplace=True)
word_freq.head()

In [ ]:
word_freq.sort_values('all', ascending=False).head(10).plot.bar(figsize=(16, 9))
plt.title('Most common words by category')
plt.ylabel('Count')
plt.xlabel('Most common words')
plt.xticks(rotation=45)
plt.show()

In [ ]:
word_freq['pct_spam'] = word_freq['spam'] / word_freq['all'] * 100
word_freq[word_freq['ham'] > 200].sort_values('pct_ham', ascending=False)

In [ ]:
ham_bigrams = pd.Series(nltk.ngrams(ham_words, 2))
top_ham_bigrams = ham_bigrams.value_counts().head(10)


In [ ]:
#Plot
top_ham_bigrams.plot.barh().invert_yaxis()
plt.xlabel('Count')
plt.ylabel('Bigram')
plt.title('Commonly occurring ham bigrams')
plt.show()